In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.sequence import pad_sequences
import shutil

In [4]:
weights_path = '/Users/prabeshsharma/Documents/Unsual_activity_Detection/Saved_Model/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Initialize the VGG16 model with the downloaded weights
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

# Define the list of class names
class_names = [
    "Abuse", "Arrest", "Arson", "Assault", "Burglary", "Explosion", 
    "Fighting", "Normal Videos", "RoadAccidents", "Robbery", 
    "Shooting", "Shoplifting", "Stealing", "Vandalism"
]



# Extract frames from video
def extract_frames(video_path, output_folder, frame_rate=10):
    video_cap = cv2.VideoCapture(video_path)
    success, image = video_cap.read()
    count = 0
    frame_count = 0

    while success:
        if count % frame_rate == 0:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.png")
            cv2.imwrite(frame_filename, image)  # Save frame as PNG file
            frame_count += 1
        success, image = video_cap.read()
        count += 1

    video_cap.release()

# Extract features from frames
def extract_features_from_frames(frames_folder, model):
    features_list = []
    for frame_file in sorted(os.listdir(frames_folder)):
        img_path = os.path.join(frames_folder, frame_file)
        
        # Read image using OpenCV
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize to target size
        img = img.astype("float32") / 255.0  # Normalize pixel values
        
        # Expand dimensions to match the input shape expected by the model
        img_array = np.expand_dims(img, axis=0)
        
        # Extract features using the model
        features = model.predict(img_array, verbose=0)
        features_list.append(features.flatten())
    
    return np.array(features_list)

# Classify video
def classify_video(features, model):
    max_length = 500  # Assuming 100 was the sequence length used during training
    padded_features = pad_sequences([features], maxlen=max_length, padding='post', dtype='float32')
        
    prediction = model.predict(padded_features)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    
    predicted_class_name = class_names[predicted_class_index]
    
    return predicted_class_name

# Cleanup frames
def cleanup_frames(frames_folder):
    shutil.rmtree(frames_folder)

# Example usage:
video_path = '/Users/prabeshsharma/Documents/Unsual_activity_Detection/2024-10-03 13.46.47.mp4'
frames_output_folder = 'ExtractedFrame'
os.makedirs(frames_output_folder, exist_ok=True)

# Step 1: Extract frames
extract_frames(video_path, frames_output_folder)

# Step 2: Extract features
features = extract_features_from_frames(frames_output_folder, feature_extractor)

# Step 3: Load the trained model
model = load_model('/Users/prabeshsharma/Documents/Unsual_activity_Detection/Saved_Model/my_model.h5')

# Step 4: Classify the video
predicted_class = classify_video(features, model)
print(f"The predicted class for the video is: {predicted_class}")

# Step 5: Cleanup frames
cleanup_frames(frames_output_folder)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
The predicted class for the video is: RoadAccidents
